In [ ]:
import pandas as pd
import numpy as np

import re

import nltk
nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:

json_file = {"intents": [
    {"tag": "greetings",
    "patterns": ["hello", "hey", "hi", "How far?", "My guy!"],
    "responses": ["Hello!", "hey!", "hey there, what can i do for you?"]
    },

    {"tag": "goodbye",
     "patterns": ["cya", "see you later", "goodbye", "got to go", "see ya!"],
     "responses": ["nice chatting with you", "talk to you soon, cheers"]
    },

    {"tag": "age",
     "patterns": ["how old are you", "what's your age?", "Age"],
     "responses": ["just a few days old, still have a lot to learn", "less than a week old"]
    },
    
    {"tag": "name",
     "patterns": ["what is your name", "what should i call you", "what's your name", "can you tell me your name?"],
     "responses": ["you can call me lisa", "I'm lisa!", "Elizabeth. but please call me lisa *winks*"]
    },

    {"tag": "shop",
     "patterns": ["I' like to buy something", "what are your products", "what do you recommend?", "what are you selling"],
     "responses": ["we sell samosa, spring rolls, chocoloate and vanilla cakes, and also bake anything you want on demand!"]
    },

    {"tag": "hours",
     "patterns": ["when are you guys open", "what are your hours", "hours of opening?"],
     "responses": ["we are open at all times", "24/7"]
    }
   ]
}

In [1]:
def flatten_json(json_file):
  tags = []
  pattern_corpus = []
  responses = []
  for intent in json_file['intents']:
    for pattern in intent['patterns']:
        pattern_corpus.append(pattern)
        tags.append(intent['tag'])
        responses.append(intent['responses'])
  if len(pattern_corpus) != len(tags):
    print('patterns do not match tags')
    raise
  data = {"tags":tags,"pattern":pattern_corpus,"responses":responses}
  return tags, pattern_corpus, responses

In [ ]:
def clean_corpus(sents):
    stemmer = nltk.stem.LancasterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    corpus = []
    for doc in sents:
        tokens = nltk.word_tokenize(doc)
        filtered_tokens = [lemmatizer.lemmatize(token) if token not in ignore_words else token for token in tokens ]
        doc = ' '.join(filtered_tokens)
        corpus.append(doc)
    return corpus

In [ ]:
def create_bow(sents):
    cv = CountVectorizer(min_df=0., max_df=1.)
    bow = cv.fit_transform(corpus)
    bow = bow.toarray()
    return bow, cv

In [ ]:
def ohe_labels(target_list):
    le = LabelEncoder()
    intent_label = le.fit_transform(target_list)

    encoder = OneHotEncoder(sparse=False) #take good note of this!! sparse set to False is the secret here

    intent_label = intent_label.reshape((-1, 1))

    intent_label = encoder.fit_transform(intent_label)
    return intent_label, le